# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port-aux-francais,-49.3500,70.2167,1.93,76,86,15.59,TF,1699318419
1,1,arkhangel'sk,64.5401,40.5433,2.85,96,100,2.12,RU,1699318420
2,2,maracaibo,10.6317,-71.6406,27.04,84,100,2.70,VE,1699318319
3,3,newman,37.3138,-121.0208,20.72,72,100,1.79,US,1699318422
4,4,fomboni,-12.2800,43.7425,27.07,76,0,7.15,KM,1699318422


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [7]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = 'Humidity',
    color = 'City'
)

# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [29]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_city_df = city_data_df.loc[city_data_df['Humidity']<=20,:]


# Drop any rows with null values
ideal_city_df = ideal_city_df.dropna()
# Display sample data
ideal_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
76,76,tamanrasset,22.7850,5.5228,16.95,16,0,1.03,DZ,1699318396
109,109,alice springs,-23.7000,133.8833,33.75,8,7,5.66,AU,1699318323
121,121,tokmok,42.8417,75.3016,11.33,20,71,1.29,KG,1699318514
176,176,timbuktu,16.7735,-3.0074,29.22,19,93,2.57,ML,1699318558
335,335,kidal,18.4411,1.4078,25.27,11,0,1.97,ML,1699318151
375,375,ghat,24.9647,10.1728,22.10,18,0,2.22,LY,1699318712
404,404,timimoun,29.2639,0.2310,21.21,14,0,1.54,DZ,1699318734
421,421,illizi,26.4833,8.4667,20.89,13,0,3.56,DZ,1699318748
452,452,alaghsas,17.0187,8.0168,24.39,15,4,4.45,NE,1699318772
462,462,ati,13.2154,18.3353,27.48,17,100,4.64,TD,1699318780


### Step 3: Create a new DataFrame called `hotel_df`.

In [30]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_city_df[['City','Country','Lat','Lng','Humidity']].copy()


# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name']=""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
76,tamanrasset,DZ,22.7850,5.5228,16,
109,alice springs,AU,-23.7000,133.8833,8,
121,tokmok,KG,42.8417,75.3016,20,
176,timbuktu,ML,16.7735,-3.0074,19,
335,kidal,ML,18.4411,1.4078,11,
375,ghat,LY,24.9647,10.1728,18,
404,timimoun,DZ,29.2639,0.2310,14,
421,illizi,DZ,26.4833,8.4667,13,
452,alaghsas,NE,17.0187,8.0168,15,
462,ati,TD,13.2154,18.3353,17,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [34]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":'accommodation.hotel',
     "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
tamanrasset - nearest hotel: فندق الأمان
alice springs - nearest hotel: Aurora Alice Springs
tokmok - nearest hotel: Жибек
timbuktu - nearest hotel: Hotel La Maison
kidal - nearest hotel: No hotel found
ghat - nearest hotel: فندق تاسيلي
timimoun - nearest hotel: أناف
illizi - nearest hotel: بوناقة
alaghsas - nearest hotel: Hôtel Zagado
ati - nearest hotel: No hotel found
yinchuan - nearest hotel: Yinchuan hotel
nguigmi - nearest hotel: Guest PAM


,City,Country,Lat,Lng,Humidity,Hotel Name
76,tamanrasset,DZ,22.7850,5.5228,16,فندق الأمان
109,alice springs,AU,-23.7000,133.8833,8,Aurora Alice Springs
121,tokmok,KG,42.8417,75.3016,20,Жибек
176,timbuktu,ML,16.7735,-3.0074,19,Hotel La Maison
335,kidal,ML,18.4411,1.4078,11,No hotel found
375,ghat,LY,24.9647,10.1728,18,فندق تاسيلي
404,timimoun,DZ,29.2639,0.2310,14,أناف
421,illizi,DZ,26.4833,8.4667,13,بوناقة
452,alaghsas,NE,17.0187,8.0168,15,Hôtel Zagado
462,ati,TD,13.2154,18.3353,17,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [55]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "CartoLight",
    frame_width = 800,
    frame_height = 600,
    size = 100,
    color = 'City',
    hover_cols=['Hotel Name','Country']
)

# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)